#  Homework 8 - Berkeley STAT 157

**Your name: XX, SID YY, teammates A,B,C** (Please add your name, SID and teammates to ease Ryan and Rachel to grade.)

**Please submit your homework through [gradescope](http://gradescope.com/)**

Handout 4/9/2019, due 4/16/2019 by 4pm.

This homework deals with sequence models for text and numbers. Due to the computational cost, we strongly encourage you to implement this on a GPU enabled machine. To make things a bit more interesting we will use a larger text collection here - [Shakespeare's collected works](http://www.gutenberg.org/files/100/100-0.txt) which are freely downloadable at Project Gutenberg. 

**This is teamwork.**

## Prerequisites - Load Data

In [38]:
import urllib3
import collections
import re
shakespeare = 'http://www.gutenberg.org/files/100/100-0.txt'

http = urllib3.PoolManager()
text = http.request('GET', shakespeare).data.decode('utf-8')
raw_dataset = ' '.join(re.sub('[^A-Za-z]+', ' ', text).lower().split())

print('number of characters: ', len(raw_dataset))
print(raw_dataset[0:70])

number of characters:  5032359
project gutenberg s the complete works of william shakespeare by willi


This dataset is quite a bit bigger than the time machine (5 million vs. 160k). For convenience we also include the remaining preprocessing steps. A bigger dataset will allow us to generate more meaningful models. 

In [39]:
idx_to_char = list(set(raw_dataset))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
# vocabulary size, i.e., the number of letters in English
vocab_size = len(char_to_idx)
# whole corpus
corpus_indices = [char_to_idx[char] for char in raw_dataset]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)
# for training
train_indices = corpus_indices[:-100000]
# for testing
test_indices = corpus_indices[-100000:]

chars: project gutenberg s 
indices: [22, 19, 21, 16, 14, 18, 9, 13, 7, 0, 9, 14, 26, 5, 14, 19, 7, 13, 1, 13]


Lastly we import other useful libraries to help you getting started.

In [40]:
import d2l
import math
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn
import time

## 1. Train Recurrent Latent Variable Models

Train a number of different latent variable models using `train_indices` to assess their performance. By default pick 256 dimensions for the hidden units. You can use the codes provided in the class. Also, we strongly encourage you to use the Gluon implementation since it's a lot faster than building it from scratch.

1. Train a single-layer RNN (with latent variables). 
1. Train a single-layer GRU.
1. Train a single-layer LSTM. 
1. Train a two-layer LSTM. 

How low can you drive the perplexity? Can you reproduce some of Shakespeare's finest writing (generate 200 characters). Start the sequence generator with `But Brutus is an honorable man`. Experiment with a number of settings:

* Number of hidden units. 
* Embedding length.
* Gradient clipping.
* Number of iterations. 
* Learning rate.

**Save** the models (at least in memory since you'll need them in the next exercise.

In [ ]:
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, ctx, idx_to_char, char_to_idx):
    state = model.begin_state(batch_size=1, ctx=ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = nd.array([output[-1]], ctx=ctx).reshape((1, 1))
        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

num_hiddens = 256
rnn_layer = rnn.RNN(num_hiddens)
rnn_layer.initialize()
num_steps = 50
num_epochs, batch_size, lr, clipping_theta = 200, 32, 1e2, 1e-2

In [ ]:
# for question 1.1: train a single-layer RNN
ctx = d2l.try_gpu()
single_rnn_model = d2l.RNNModel(rnn_layer, vocab_size)
single_rnn_model.initialize(force_reinit=True, ctx = ctx)
pred_period, pred_len, prefixes = 50, 50, ['but brutus is an honorable man']
d2l.train_and_predict_rnn_gluon(single_rnn_model, num_hiddens, vocab_size, ctx, train_indices, idx_to_char,\
                                char_to_idx, num_epochs, num_steps, lr, clipping_theta, batch_size,\
                                pred_period, pred_len, prefixes)

epoch 50, perplexity 3.801209, time 15.28 sec
 - but brutus is an honorable man s son s son s son s son s son s son s son s son s
epoch 100, perplexity 3.799337, time 14.86 sec
 - but brutus is an honorable man and they are the servant they are the servant the
epoch 150, perplexity 3.806173, time 15.37 sec
 - but brutus is an honorable man and they shall be the servant they shall be the s


In [ ]:
predict_rnn_gluon('but brutus is an honorable man', 200, single_rnn_model, vocab_size, ctx,\
                  idx_to_char, char_to_idx)

In [ ]:
# for question 1.2, single layer lstm
lstm_layer = rnn.LSTM(num_hiddens)
single_lstm_model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_gluon(single_lstm_model, num_hiddens, vocab_size, ctx, train_indices, idx_to_char,\
                                char_to_idx, num_epochs, num_steps, lr, clipping_theta, batch_size,\
                                pred_period, pred_len, prefixes)

In [ ]:
predict_rnn_gluon('but brutus is an honorable man', 200, single_lstm_model, vocab_size, ctx,\
                  idx_to_char, char_to_idx)

In [ ]:
# for question 1.3, single layer gru
gru_layer = rnn.GRU(num_hiddens)
single_gru_model = d2l.RNNModel(gru_layer, vocab_size)
d2l.train_and_predict_rnn_gluon(single_gru_model, num_hiddens, vocab_size, ctx, train_indices, idx_to_char,\
                                char_to_idx, num_epochs, num_steps, lr, clipping_theta, batch_size,\
                                pred_period, pred_len, prefixes)

In [ ]:
predict_rnn_gluon('but brutus is an honorable man', 200, single_gru_model, vocab_size, ctx,\
                  idx_to_char, char_to_idx)

In [ ]:
# for question 1.4, double layers lstm
lstm_layer = rnn.LSTM(num_hiddens, num_layers=2)
double_lstm_model = d2l.RNNModel(lstm_layer, vocab_size)
d2l.train_and_predict_rnn_gluon(double_lstm_model, num_hiddens, vocab_size, ctx, train_indices, idx_to_char,\
                                char_to_idx, num_epochs, num_steps, lr, clipping_theta, batch_size,\
                                pred_period, pred_len, prefixes)

In [ ]:
predict_rnn_gluon('but brutus is an honorable man', 200, double_lstm_model, vocab_size, ctx,\
                  idx_to_char, char_to_idx)

## 2. Test Error

So far we measured perplexity only on the training set. 

1. Implement a perplexity calculator that does not involve training.
1. Compute the perplexity of the best models in each of the 4 categories on the test set. By how much does it differ? 

In [34]:
# for question 2
# This function is used to test perplexity on test set, not including training
def test_perplexity(model, ctx, corpus_indices, idx_to_char, char_to_idx, num_epochs, num_steps, batch_size):
    for epoch in range(num_epochs):
        loss = gloss.SoftmaxCrossEntropyLoss()
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(
            corpus_indices, batch_size, num_steps, ctx)
        state = model.begin_state(batch_size=batch_size, ctx=ctx)
        for X, Y in data_iter:
            for s in state:
                s.detach()
            with autograd.record():
                (output, state) = model(X, state)
                y = Y.T.reshape((-1,))
                l = loss(output, y).mean()
            l_sum += l.asscalar() * y.size
            n += y.size
        print('epoch %d, perplexity %f, time %.2f sec' % (
            epoch + 1, math.exp(l_sum / n), time.time() - start))

In [35]:
num_epochs, num_steps, batch_size = 5, 50, 1

In [ ]:
# test perplexity on gru
test_perplexity(single_rnn_model, ctx, test_indices, idx_to_char, char_to_idx, num_epochs, num_steps, batch_size)

In [37]:
# test perplexity on single lstm
test_perplexity(single_lstm_model, ctx, test_indices, idx_to_char, char_to_idx, num_epochs, num_steps, batch_size)

epoch 1, perplexity 16.582907, time 5.66 sec


KeyboardInterrupt: 

In [36]:
# test perplexity on gru
test_perplexity(single_gru_model, ctx, test_indices, idx_to_char, char_to_idx, num_epochs, num_steps, batch_size)

epoch 1, perplexity 1.285600, time 3.68 sec
epoch 2, perplexity 1.285531, time 3.45 sec
epoch 3, perplexity 1.284768, time 3.44 sec
epoch 4, perplexity 1.284500, time 3.47 sec
epoch 5, perplexity 1.285531, time 3.46 sec
epoch 6, perplexity 1.284853, time 3.57 sec
epoch 7, perplexity 1.285207, time 3.37 sec
epoch 8, perplexity 1.285309, time 3.36 sec
epoch 9, perplexity 1.285423, time 3.56 sec
epoch 10, perplexity 1.284604, time 3.52 sec


In [ ]:
# test perplexity on double lstm
test_perplexity(double_lstm_model, ctx, test_indices, idx_to_char, char_to_idx, num_epochs, num_steps, batch_size)

## 3. $N$-Gram Model

So far we only considered latent variable models. Let's see what happens if we use a regular $N$-gram model and an autoregressive setting. That is, we aim to predict the next character given the current characters one character at a time. For this implement the following:

1. Split data into $(x,y)$ pairs as before, just that we now use very short subsequences, e.g. only $5$ characters. That is, `But Brutus` turns into the tuples (`(But B, r)`, `(ut Br, u)`, `(t Bru, t)`, `( Brut, u)`, `(Brutu, s)`). 
1. Use one-hot encoding for each character separately and combine them all. 
    * In one case use a sequential encoding to obtain an embedding proportional to the length of the sequence.
    * Use a bag of characters encoding that sums over all occurrences.
1. Implement an MLP with one hidden layer and 256 hidden units.
1. Train it to output the next character.

How accurate is the model? How does the number of operations and weights compare to an RNN, a GRU and an LSTM discussed above?